In [ ]:
import os
import tempfile
import shutil
import pathlib

import time_machine

# import to register magics
import sqlmesh

In [ ]:
freezer = time_machine.travel("2032-01-01 00:00:00 UTC")
freezer.start()

In [ ]:
starting_dir = pathlib.Path(os.getcwd())
prev_parent_stem = ""
root_dir = None
for parent in starting_dir.parents:
    if parent.stem == "sqlmesh" and prev_parent_stem == "tests":
        root_dir = parent
        break
    prev_parent_stem = parent.stem
else:
    raise RuntimeError("Couldn't find root dir")

EXAMPLE_SUSHI_DIR = pathlib.Path(root_dir) / "examples" / "sushi"
str(EXAMPLE_SUSHI_DIR)

In [ ]:
TARGET_SUSHI_DIR = pathlib.Path(tempfile.gettempdir()) / "test_jupyter_notebook"
str(TARGET_SUSHI_DIR)

In [ ]:
TARGET_MODEL_DIR = TARGET_SUSHI_DIR / "models"
str(TARGET_MODEL_DIR)

In [ ]:
def ignore(src, names):
    if pathlib.Path(src).name in {".cache", "__pycache__", "logs", "data"}:
        return names
    return []

if TARGET_SUSHI_DIR.exists():
    shutil.rmtree(str(TARGET_SUSHI_DIR))
shutil.copytree(str(EXAMPLE_SUSHI_DIR), str(TARGET_SUSHI_DIR), ignore=ignore)

In [ ]:
%context $TARGET_SUSHI_DIR

### Leaving plan unapplied to verify that the text boxes are printed

In [ ]:
%plan

In [ ]:
%plan --no-prompts --auto-apply

In [ ]:
%plan dev --no-prompts --auto-apply --include-unmodified

In [ ]:
%render sushi.top_waiters

In [ ]:
%render sushi.top_waiters --no-format

In [ ]:
%evaluate sushi.top_waiters

In [ ]:
top_waiters = TARGET_MODEL_DIR / "top_waiters.sql"
top_waiters.write_text("""/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  audits (
    unique_values(columns=[waiter_id])
  ),
  grain waiter_id
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1 as new_col
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 10
""")

In [ ]:
%diff prod

In [ ]:
%plan dev --no-prompts --auto-apply

In [ ]:
%dag

In [ ]:
%migrate

In [ ]:
%table_diff prod:dev --model sushi.top_waiters

In [ ]:
%run_test {TARGET_SUSHI_DIR}/tests/test_customer_revenue_by_day.yaml::test_customer_revenue_by_day

In [ ]:
%audit sushi.top_waiters

In [ ]:
%%fetchdf my_result
SELECT 1

In [ ]:
my_result

In [ ]:
%info

In [ ]:
%migrate

In [ ]:
freezer = time_machine.travel("2032-01-02 00:00:00")
freezer.start()

In [ ]:
%run_dag

In [ ]:
%invalidate dev

In [1]:
%context

UsageError: Line magic function `%context` not found.


In [2]:
import sqlmesh

In [27]:
%create_test?

Docstring:
::

  %create_test --query QUERY [QUERY ...] [--overwrite]
                   [--var VAR [VAR ...]] [--path PATH] [--name NAME]
                   model

Generate a unit test fixture for a given model.

positional arguments:
  model

options:
  --query <QUERY [QUERY ...]>, -q <QUERY [QUERY ...]>
                        Queries that will be used to generate data for the
                        model's dependencies.
  --overwrite, -o       When true, the fixture file will be overwritten in
                        case it already exists.
  --var <VAR [VAR ...]>, -v <VAR [VAR ...]>
                        Key-value pairs that will define variables needed by
                        the model.
  --path PATH, -p PATH  The file path corresponding to the fixture, relative
                        to the test directory. By default, the fixture will be
                        created under the test directory and the file name
                        will be inferred based on the test's 

In [9]:
%lsmagic?

Docstring: List currently available magic functions.
File:      ~/.pyenv/versions/3.11.5/envs/sqlmesh311/lib/python3.11/site-packages/IPython/core/magics/basic.py